In [16]:
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.tools import DuckDuckGoSearchResults
from langchain.retrievers import WikipediaRetriever
from langchain.schema import SystemMessage

llm = ChatOpenAI(temperature=0.1)


class SearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for contents.")


class StoreToTextToolArgsSchema(BaseModel):
    name: str = Field(description="The name of file to be stored")
    contents: str = Field(description="The contents will stroing in .txt file.")


class WebSearchTool(BaseTool):
    name = "WebSearchTool"
    description = """
    Use this tool if you want to retrieve research information from web using DuckDuckGo for a specific keyword.
    It takes a query as an argument.
    
    """
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults(backend="auto")
        return ddg.run(query)


class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = """
    Use this tool if you want to retrieve research information from wikipedia for a specific keyword.
    It takes a query as an argument.
    
    """
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query):
        retriever = WikipediaRetriever(top_k_results=1)
        return retriever.get_relevant_documents(query)


class StoreToTextTool(BaseTool):
    name = "StoreToTextTool"
    description = """
    Use this tool if you want to store contents into <name>.txt.
    It takes a name and contents as an argument.
    
    """
    args_schema: Type[StoreToTextToolArgsSchema] = StoreToTextToolArgsSchema

    def _run(self, name, contents):
        with open(f"{name}.txt", "w") as file:
            file.write(contents)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WebSearchTool(),
        WikipediaSearchTool(),
        StoreToTextTool(),
    ],
    agent_kwargs={
        "system_message": SystemMessage(
            content="""
You are a expert researcher.

you should try to search in Wikipedia or DuckDuckGo.
If it finds a website in DuckDuckGo it should enter the website and extract it's content.
and then it should finish by saving the research to a .txt file.
"""
        )
    },
)

prompt = "Research about the XZ backdoor"
agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `WebSearchTool` with `{'query': 'XZ backdoor'}`


[snippet: What Does the Backdoor Do? Malicious code added to xz Utils versions 5.6.0 and 5.6.1 modified the way the software functions. The backdoor manipulated sshd, the executable file used to make remote ..., title: The XZ Backdoor: Everything You Need to Know - WIRED, link: https://www.wired.com/story/xz-backdoor-everything-you-need-to-know/], [snippet: Because the backdoor was discovered before the malicious versions of xz Utils were added to production versions of Linux, "it's not really affecting anyone in the real world," Will Dormann, a ..., title: Backdoor found in widely used Linux utility targets encrypted SSH ..., link: https://arstechnica.com/security/2024/03/backdoor-found-in-widely-used-linux-utility-breaks-encrypted-ssh-connections/], [snippet: The back door means that the connection is no longer private and could allow an attacker to insert their own text commands into

{'input': 'Research about the XZ backdoor',
 'output': 'I have researched about XZ Utils and here is a summary of the information:\n\nXZ Utils (previously LZMA Utils) is a set of free software command-line lossless data compressors, including the programs lzma and xz, for Unix-like operating systems and Microsoft Windows. It uses the Lempel–Ziv–Markov chain algorithm (LZMA) for compression/decompression. XZ Utils can compress and decompress the xz and lzma file formats, achieving higher compression rates than alternatives like zip, gzip, and bzip2.\n\nThe software consists of two major components: xz (command-line compressor and decompressor) and liblzma (a software library with an API similar to zlib). XZ supports multi-threaded compression and allows preprocessing filters to improve compression.\n\nDevelopment of XZ Utils took place within the Tukaani Project, and the .xz file format specification was officially released in January 2009. The source code for xz and liblzma has been re